In [1]:
import yaml
import torch
import time


from models import GAT, GCN, SWEGNN
from models.swe_gnn_ned import SWEGNN_NED
from data import TemporalGraphDataset

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)

dataset, info = TemporalGraphDataset(node_features=config['node_features'],
                    edge_features=config['edge_features'],
                    **config['dataset_parameters']).load()

In [4]:
print(dataset[0])
print(info)

Data(x=[1268, 6], edge_index=[2, 2612], edge_attr=[2612, 8], y=[1268, 1], pos=[2, 1268])
{'num_static_node_features': 3, 'num_dynamic_node_features': 1, 'num_static_edge_features': 5, 'num_dynamic_edge_features': 1, 'previous_timesteps': 2}


In [5]:
num_train = int(len(dataset) * 0.8) # 80% train, 20% test

train_dataset = dataset[:num_train]
# train_loader = DataLoader(train_dataset) # batch_size=32, shuffle=True

test_dataset = dataset[num_train:]
# test_loader = DataLoader(test_dataset) # batch_size=32, shuffle=True

In [6]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
base_model_params = {
    'static_node_features': info['num_static_node_features'],
    'dynamic_node_features': info['num_dynamic_node_features'],
    'static_edge_features': info['num_static_edge_features'],
    'dynamic_edge_features': info['num_dynamic_edge_features'],
    'previous_timesteps': info['previous_timesteps'],
    'device': device,
}
lr_info = config['training_parameters']

In [7]:
def train(model, loss_func, optimizer):
    start_time = time.time()
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for graph in train_dataset:
            graph = graph.to(device)
            labels = graph.y

            optimizer.zero_grad()

            outputs = model(graph)
            loss = loss_func(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / num_train
        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {epoch_loss:.4f}')
    end_time = time.time()
    print(f'Total training time: {end_time - start_time} seconds')


def test(model, loss_func):
    start_time = time.time()
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for graph in test_dataset:
            graph = graph.to(device)
            labels = graph.y

            outputs = model(graph)
            loss = loss_func(outputs, labels)
            running_loss += loss.item()
    end_time = time.time()

    # Print validation statistics
    print(f'Validation Loss: {running_loss:.4f}')
    print(f'Inference time: {end_time - start_time} seconds')

In [8]:
gcn_params = config['GCN']
model = GCN(**gcn_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()

train(model, loss_func, optimizer)

Epoch [1/10], Training Loss: 1.2887
Epoch [2/10], Training Loss: 0.0256
Epoch [3/10], Training Loss: 0.0227
Epoch [4/10], Training Loss: 0.0211
Epoch [5/10], Training Loss: 0.0210
Epoch [6/10], Training Loss: 0.0213
Epoch [7/10], Training Loss: 0.0208
Epoch [8/10], Training Loss: 0.0208
Epoch [9/10], Training Loss: 0.0203
Epoch [10/10], Training Loss: 0.0200
Total training time: 42.38818407058716 seconds


In [9]:
test(model, loss_func)

Validation Loss: 0.2097
Inference time: 0.6375396251678467 seconds


In [12]:
gat_params = config['GCN']
model = GAT(**gat_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()

train(model, loss_func, optimizer)

Epoch [1/10], Training Loss: 1.6340
Epoch [2/10], Training Loss: 0.0280
Epoch [3/10], Training Loss: 0.0269
Epoch [4/10], Training Loss: 0.0315
Epoch [5/10], Training Loss: 0.0221
Epoch [6/10], Training Loss: 0.0215
Epoch [7/10], Training Loss: 0.0522
Epoch [8/10], Training Loss: 0.0214
Epoch [9/10], Training Loss: 0.0206
Epoch [10/10], Training Loss: 0.0206
Total training time: 69.85159826278687 seconds


In [13]:
test(model, loss_func)

Validation Loss: 0.1827
Inference time: 0.7385728359222412 seconds


In [14]:
swe_gnn_params = config['SWEGNN']
model = SWEGNN(**swe_gnn_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()
train(model, loss_func, optimizer)

Epoch [1/10], Training Loss: 0.0832
Epoch [2/10], Training Loss: 0.0206
Epoch [3/10], Training Loss: 0.0206
Epoch [4/10], Training Loss: 0.0206
Epoch [5/10], Training Loss: 0.0208
Epoch [6/10], Training Loss: 0.0206
Epoch [7/10], Training Loss: 0.0206
Epoch [8/10], Training Loss: 0.0206
Epoch [9/10], Training Loss: 0.0207
Epoch [10/10], Training Loss: 0.0206
Total training time: 184.67357468605042 seconds


In [15]:
test(model, loss_func)

Validation Loss: 0.1828
Inference time: 1.2828199863433838 seconds


In [16]:
swe_gnn_params = config['SWEGNN']
if 'hidden_features' in swe_gnn_params:
    del swe_gnn_params['hidden_features']
model = SWEGNN_NED(**swe_gnn_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()
train(model, loss_func, optimizer)

Epoch [1/10], Training Loss: 2.8461
Epoch [2/10], Training Loss: 0.0378
Epoch [3/10], Training Loss: 0.0400
Epoch [4/10], Training Loss: 0.0330
Epoch [5/10], Training Loss: 0.0319
Epoch [6/10], Training Loss: 0.0281
Epoch [7/10], Training Loss: 0.0237
Epoch [8/10], Training Loss: 0.0228
Epoch [9/10], Training Loss: 0.0210
Epoch [10/10], Training Loss: 0.0208
Total training time: 130.09719252586365 seconds


In [17]:
test(model, loss_func)

Validation Loss: 0.1876
Inference time: 1.0696780681610107 seconds


## Self-Supervised Learning Methods